In [2]:
# AES S-box (Substitution Box)
AES_SBOX = [
    0x63, 0x7C, 0x77, 0x7B, 0xF2, 0x6B, 0x6F, 0xC5, 0x30, 0x01, 0x67, 0x2B, 0xFE, 0xD7, 0xAB, 0x76,
    0xCA, 0x82, 0xC9, 0x7D, 0xFA, 0x59, 0x47, 0xF0, 0xAD, 0xD4, 0xA2, 0xAF, 0x9C, 0xA4, 0x72, 0xC0,
    0xB7, 0xFD, 0x93, 0x26, 0x36, 0x3F, 0xF7, 0xCC, 0x34, 0xA5, 0xE5, 0xF1, 0x71, 0xD8, 0x31, 0x15,
    0x04, 0xC7, 0x23, 0xC3, 0x18, 0x96, 0x05, 0x9A, 0x07, 0x12, 0x80, 0xE2, 0xEB, 0x27, 0xB2, 0x75,
    0x09, 0x83, 0x2C, 0x1A, 0x1B, 0x6E, 0x5A, 0xA0, 0x52, 0x3B, 0xD6, 0xB3, 0x29, 0xE3, 0x2F, 0x84,
    0x53, 0xD1, 0x00, 0xED, 0x20, 0xFC, 0xB1, 0x5B, 0x6A, 0xCB, 0xBE, 0x39, 0x4A, 0x4C, 0x58, 0xCF,
    0xD0, 0xEF, 0xAA, 0xFB, 0x43, 0x4D, 0x33, 0x85, 0x45, 0xF9, 0x02, 0x7F, 0x50, 0x3C, 0x9F, 0xA8,
    0x51, 0xA3, 0x40, 0x8F, 0x92, 0x9D, 0x38, 0xF5, 0xBC, 0xB6, 0xDA, 0x21, 0x10, 0xFF, 0xF3, 0xD2,
    0xCD, 0x0C, 0x13, 0xEC, 0x5F, 0x97, 0x44, 0x17, 0xC4, 0xA7, 0x7E, 0x3D, 0x64, 0x5D, 0x19, 0x73,
    0x60, 0x81, 0x4F, 0xDC, 0x22, 0x2A, 0x90, 0x88, 0x46, 0xEE, 0xB8, 0x14, 0xDE, 0x5E, 0x0B, 0xDB,
    0xE0, 0x32, 0x3A, 0x0A, 0x49, 0x06, 0x24, 0x5C, 0xC2, 0xD3, 0xAC, 0x62, 0x91, 0x95, 0xE4, 0x79,
    0xE7, 0xC8, 0x37, 0x6D, 0x8D, 0xD5, 0x4E, 0xA9, 0x6C, 0x56, 0xF4, 0xEA, 0x65, 0x7A, 0xAE, 0x08,
    0xBA, 0x78, 0x25, 0x2E, 0x1C, 0xA6, 0xB4, 0xC6, 0xE8, 0xDD, 0x74, 0x1F, 0x4B, 0xBD, 0x8B, 0x8A,
    0x70, 0x3E, 0xB5, 0x66, 0x48, 0x03, 0xF6, 0x0E, 0x61, 0x35, 0x57, 0xB9, 0x86, 0xC1, 0x1D, 0x9E,
    0xE1, 0xF8, 0x98, 0x11, 0x69, 0xD9, 0x8E, 0x94, 0x9B, 0x1E, 0x87, 0xE9, 0xCE, 0x55, 0x28, 0xDF,
    0x8C, 0xA1, 0x89, 0x0D, 0xBF, 0xE6, 0x42, 0x68, 0x41, 0x99, 0x2D, 0x0F, 0xB0, 0x54, 0xBB, 0x16
]

# AES Round Key Generation
def aes_key_expansion(key):
    # Rijndael Rcon
    rcon = [0x01, 0x02, 0x04, 0x08, 0x10, 0x20, 0x40, 0x80, 0x1B, 0x36]

    # Initialize round keys
    nk = len(key) // 4  # Number of 32-bit words in key
    nr = nk + 6         # Number of rounds

    w = [0] * (4 * (nr + 1))

    # Copy initial key
    for i in range(nk):
        w[i] = (key[4*i] << 24) | (key[4*i+1] << 16) | (key[4*i+2] << 8) | key[4*i+3]

    # Key expansion
    for i in range(nk, 4 * (nr + 1)):
        temp = w[i-1]

        if i % nk == 0:
            # RotWord + SubWord + Rcon
            temp = ((temp & 0xFFFFFF) << 8) | (temp >> 24)
            temp = (AES_SBOX[(temp >> 24) & 0xFF] << 24) | \
                   (AES_SBOX[(temp >> 16) & 0xFF] << 16) | \
                   (AES_SBOX[(temp >> 8) & 0xFF] << 8) | \
                   AES_SBOX[temp & 0xFF]
            temp ^= (rcon[i//nk - 1] << 24)
        elif nk > 6 and i % nk == 4:
            # SubWord
            temp = (AES_SBOX[(temp >> 24) & 0xFF] << 24) | \
                   (AES_SBOX[(temp >> 16) & 0xFF] << 16) | \
                   (AES_SBOX[(temp >> 8) & 0xFF] << 8) | \
                   AES_SBOX[temp & 0xFF]

        w[i] = w[i - nk] ^ temp

    # Convert to round keys
    round_keys = []
    for i in range(nr + 1):
        round_key = []
        for j in range(4):
            word = w[4*i + j]
            round_key.extend([(word >> 24) & 0xFF, (word >> 16) & 0xFF,
                             (word >> 8) & 0xFF, word & 0xFF])
        round_keys.append(round_key)

    return round_keys

# Example usage
key = [0x2b, 0x7e, 0x15, 0x16, 0x28, 0xae, 0xd2, 0xa6,
       0xab, 0xf7, 0x15, 0x88, 0x09, 0xcf, 0x4f, 0x3c]
round_keys = aes_key_expansion(key)
for i, rk in enumerate(round_keys):
    print(f"Round {i:2}: {' '.join(f'{b:02x}' for b in rk)}")

Round  0: 2b 7e 15 16 28 ae d2 a6 ab f7 15 88 09 cf 4f 3c
Round  1: a0 fa fe 17 88 54 2c b1 23 a3 39 39 2a 6c 76 05
Round  2: f2 c2 95 f2 7a 96 b9 43 59 35 80 7a 73 59 f6 7f
Round  3: 3d 80 47 7d 47 16 fe 3e 1e 23 7e 44 6d 7a 88 3b
Round  4: ef 44 a5 41 a8 52 5b 7f b6 71 25 3b db 0b ad 00
Round  5: d4 d1 c6 f8 7c 83 9d 87 ca f2 b8 bc 11 f9 15 bc
Round  6: 6d 88 a3 7a 11 0b 3e fd db f9 86 41 ca 00 93 fd
Round  7: 4e 54 f7 0e 5f 5f c9 f3 84 a6 4f b2 4e a6 dc 4f
Round  8: ea d2 73 21 b5 8d ba d2 31 2b f5 60 7f 8d 29 2f
Round  9: ac 77 66 f3 19 fa dc 21 28 d1 29 41 57 5c 00 6e
Round 10: d0 14 f9 a8 c9 ee 25 89 e1 3f 0c c8 b6 63 0c a6
